In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import norm
import requests
import re
from bs4 import BeautifulSoup
import time
from collections import Counter
import datetime
import os
from urllib.request import urlopen, Request

# fake a header of a browser
headers = {'User-Agent': 'Chrome/39.0.2171.95'}

# Earning data from yahoo

In [3]:
ticker = "UBER"

the_url = "https://finance.yahoo.com/calendar/earnings?symbol={0}".format(ticker)
# fake a header of a browser
response = requests.get(the_url, headers=headers)
# beatifulsoup find all the right tags
soup = BeautifulSoup(response.text, features='lxml')

In [4]:
eraning_time=soup.find_all(attrs={"aria-label": "Earnings Date"})
est_eps=soup.find_all(attrs={"aria-label": "EPS Estimate"})
rpt_eps=soup.find_all(attrs={"aria-label": "Reported EPS"})
surprise =soup.find_all(attrs={"aria-label": "Surprise(%)"})

In [125]:
earning_release_time = list(map(lambda x:pd.to_datetime(x.text[:-3]),eraning_time))
estimated_eps = list(map(lambda x:x.text,est_eps))
reported_eps = list(map(lambda x:x.text,rpt_eps))
surprise_percentage = list(map(lambda x:x.text,surprise))

In [149]:
earning_release = pd.DataFrame([earning_release_time,estimated_eps,reported_eps,surprise_percentage]).T
earning_release.columns = ["EarningsDate","EstimateEPS","ReportedEPS","Surprise"]
earning_release.set_index("EarningsDate",drop=True,inplace=True)
earning_release.replace("N/A",np.nan,inplace=True)
earning_release = earning_release.astype(float)

# Scrap the earning news from seeking alpha

In [5]:
the_url = "https://seekingalpha.com/symbol/{0}".format(ticker)
# fake a header of a browser
headers = {'User-Agent': 'Chrome/39.0.2171.95'}
response = requests.get(the_url, headers=headers)
# beatifulsoup find all the right tags
soup = BeautifulSoup(response.text, features='lxml')
head_url ="https://seekingalpha.com/"

In [72]:
key_url=[]
key_words = "eps"
inews=soup.find_all(attrs={"sasource": re.compile("qp_latest")})
[key_url.append(i.get('href')) for i in inews if key_words in i.get('href')]
if len(key_url) != 0:
    iurl = key_url[0]
    response = requests.get(head_url+, headers=headers)
    soup = BeautifulSoup(response.text, features='lxml')

['/news/3571490-uber-eps-misses-0_80-beats-on-revenue']

In [61]:
inews=soup.find_all(href=re.compile("eps"))
inews[-1].get('href')

'/news/3571490-uber-eps-misses-0_80-beats-on-revenue'

In [42]:
inews=soup.find_all("div",class_="symbol_article",)
for i in inews:
    i.text

# Scrape the headlines news

In [5]:
the_url = "https://seekingalpha.com/market-news"
# fake a header of a browser
headers = {'User-Agent': 'Chrome/39.0.2171.95'}
response = requests.get(the_url, headers=headers)
# beatifulsoup find all the right tags
soup = BeautifulSoup(response.text, features='lxml')

In [46]:
live_news_sa=soup.find_all(attrs={"sasource": re.compile("market_news_all")})
comment_news_sa=soup.find_all(attrs={"sasource": re.compile("market_news_commentary")})
inews=soup.find_all(attrs={"sasource": re.compile("market_news_h")})
for i in inews:
    print(i.text)

In [48]:
key_word = "buy" #["buy","exit","cut"]
the_url = "https://seekingalpha.com/market-news"
# fake a header of a browser
response = requests.get(the_url, headers=headers)
# beatifulsoup find all the right tags
soup = BeautifulSoup(response.text, features='lxml')
inews = soup.find_all(href=re.compile(key_word))
# if key word is in the website address
key_url = [i.get('href') for i in inews]

['/news/3575345-icahn-buys-delek-boost-stake-in-occidental',
 '/news/3575343-appaloosa-management-buys-twitter-exits-caesars-cuts-facebook']

# Earnings incoming companies

In [4]:
recent_day = 3
today_date = pd.to_datetime(datetime.date.today())
target_date = pd.to_datetime(datetime.date.today()+datetime.timedelta(days = recent_day))
target_date

all_df =  pd.DataFrame()
for i in range(1,30):
    the_url = f'https://seekingalpha.com/earnings/earnings-calendar/{i}'
    response = requests.get(the_url, headers=headers)
    # beatifulsoup find all the right tags
    soup = BeautifulSoup(response.text, features='lxml')
    tic = list(map(lambda x:x.text,soup.find_all('a',class_='sym')))
    date = list(map(lambda x:x.text,soup.find_all('span',class_='release-date')))
    time = list(map(lambda x:x.text,soup.find_all('span',class_='release-time')))
    name = list(map(lambda x:x.text,soup.find_all('span',class_='ticker-name')))
    df = pd.DataFrame([tic,name,date,time]).T
    all_df = pd.concat([all_df,df],axis=0)
    if pd.to_datetime(all_df.iloc[-1,2])>target_date:break


all_df.columns=["Ticker","Name","Date","Time"]
all_df.index = all_df.Ticker
all_df.Date = pd.to_datetime(all_df.Date)
all_df = all_df[all_df.Date <=  target_date].sort_values(by="Date")

In [5]:
sp_names = pd.read_csv("dictionary\\SP500.csv",index_col=0)
all_df_s = all_df.join(sp_names,how ="inner").iloc[:,[1,2,3,7]]

rs1000_names = pd.read_csv("dictionary\\RU1000.csv",index_col=0)
all_df_r1 = all_df.join(rs1000_names,how ="inner").iloc[:,:]

cn_names = pd.read_csv("dictionary\\CN.csv",index_col=0)
all_df_c = all_df.join(cn_names,how ="inner").iloc[:,[1,2,3,8]].sort_values(by="Date")

rs3000_names = pd.read_csv("dictionary\\RU3000.csv",index_col=0)
all_df_r3 = all_df.join(rs3000_names,how ="inner").iloc[:,:]

In [6]:
exp = pd.read_csv("examples\\Public_earnings_dates_May2020.csv")
exp.index = list(map(lambda x:x.split(" ")[0],exp.Ticker))

FileNotFoundError: [Errno 2] File b'example\\Public_earnings_dates_May2020.csv' does not exist: b'example\\Public_earnings_dates_May2020.csv'

In [ ]:
temp_all = all_df_r3.join(exp,how="outer",lsuffix='_left', rsuffix='_right')

In [ ]:
temp_all.fillna(0,inplace = True)

temp_all[temp_all.Ticker_left ==0].index.to_numpy()

In [9]:
temp_all[temp_all.Ticker_right ==0].index.to_numpy()

array(['ADVM', 'AMEH', 'AVRO', 'CVGI', 'HALL', 'HBB', 'HFFG', 'HUD',
       'LGF.A', 'M', 'NWFL', 'OLP', 'PGR', 'RCL', 'RDI', 'RJF', 'ROSE',
       'RRTS', 'RTW', 'SENEA', 'SIEB', 'SPN', 'SRT', 'USAT', 'WSM'],
      dtype=object)

# Earning names from YAHOO

In [9]:
recent_day = 5
today_date = str(pd.to_datetime(datetime.date.today()).date())
target_date = str(pd.to_datetime(datetime.date.today()+datetime.timedelta(days = recent_day)).date())

In [12]:
the_url = f'https://finance.yahoo.com/calendar/earnings?&day={target_date}'
response = requests.get(the_url, headers=headers)
# beatifulsoup find all the right tags
soup = BeautifulSoup(response.text, features='lxml')

In [13]:
tic =soup.find_all(attrs={"aria-label": "Symbol"})
name =soup.find_all(attrs={"aria-label": "Company"})
earning_call_time=soup.find_all(attrs={"aria-label": "Earnings Call Time"})
eps_est =soup.find_all(attrs={"aria-label": "Surprise(%)"})

In [45]:
tic =soup.find_all("td")
tic

[]

In [40]:
the_url = f'https://finance.yahoo.com/calendar/earnings?&day={target_date}'

response = requests.get(the_url, headers=headers)

page = response.text.replace("<!-->", "")
soup = BeautifulSoup(response.text, "lxml")